<a href="https://colab.research.google.com/github/hainesdata/gas/blob/main/lr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
# Imports
import pandas as pd
import plotly.express as px
import numpy as np
import datetime
import requests
import regex as re
import random
import time
import sys
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from statistics import stdev, mean, median, variance
from math import sqrt
from bs4 import BeautifulSoup

In [3]:
# Init (Code 1)
raw = pd.read_csv('gas_buddy_2022-04-18.csv')
raw.info()
raw.nunique()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53737 entries, 0 to 53736
Data columns (total 23 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   services_included  53737 non-null  object 
 1   price_time_stamp   53737 non-null  object 
 2   currency           53737 non-null  object 
 3   postal_code        53699 non-null  object 
 4   loc_name           53737 non-null  object 
 5   city               53737 non-null  object 
 6   review_count       52907 non-null  float64
 7   state              53737 non-null  object 
 8   zip_code_searched  53737 non-null  int64  
 9   latitude           53737 non-null  float64
 10  product_name       53737 non-null  object 
 11  payment_type       53737 non-null  object 
 12  DATE_SCRAPED       53737 non-null  object 
 13  RUN_START_DATE     53737 non-null  object 
 14  source_url         53737 non-null  object 
 15  phone              50483 non-null  object 
 16  loc_number         537

services_included     2932
price_time_stamp     11960
currency                 1
postal_code           8508
loc_name              1292
city                  1010
review_count           469
state                    2
zip_code_searched      547
latitude              9459
product_name             6
payment_type             2
DATE_SCRAPED          1472
RUN_START_DATE           1
source_url            9471
phone                 8605
loc_number            9471
price_current          283
country                  1
longitude             9460
address_1             9446
address_2             5826
overall_rating          40
dtype: int64

In [ ]:
# Exploratory (Code 2 - 18)

# CODE 2----------------------------------------------------------------
# Create working copy of raw data to preserve raw data
sandbox = raw.copy()

# Instantiate LabelEncoder and StandardScaler objects from scikit-learn
le = LabelEncoder()
s = StandardScaler()

# Encode
sandbox['services_included'] = le.fit_transform(sandbox['services_included'])
si_1d = np.array(sandbox['services_included']).reshape(-1, 1)

# Standardize
s.fit(si_1d)
sandbox['services_included'] = s.transform(si_1d)

# Return distribution histogram
px.histogram(x=sandbox['services_included'], height=300, width=500).show()

# CODE 3----------------------------------------------------------------
sandbox = sandbox[sandbox['price_current'] != 0]
px.box(sandbox, x='payment_type', y='price_current', width=400, height=500).show()

# CODE 4----------------------------------------------------------------
px.histogram(x=sandbox['overall_rating'], height=300, width=500).show()

# CODE 5----------------------------------------------------------------
px.histogram(x=sandbox['review_count'], height=300, width=500).show()


# CODE 6----------------------------------------------------------------
le.fit(sandbox[['loc_number']])
sandbox['loc_number'] = le.transform(sandbox[['loc_number']])
px.histogram(x=sandbox['loc_number'], height=300, width=500).show()


# CODE 7----------------------------------------------------------------
le.fit(sandbox[['loc_name']])
sandbox['loc_name'] = le.transform(sandbox[['loc_name']])
px.histogram(x=sandbox['loc_name'], height=300, width=500).show()

# CODE 8----------------------------------------------------------------
print(len(sandbox['city'].unique()))

le.fit(sandbox[['city']])
sandbox['city'] = le.transform(sandbox[['city']])
px.histogram(x=sandbox['city'], height=300, width=500).show()

# CODE 9-18--------------------------------------------------------------
px.box(sandbox, x='product_name', y='price_current', width=600).show()

px.histogram(sandbox, x='price_current', color='product_name', barmode='stack', nbins=64, width=1000).show()

px.scatter(sandbox, x='review_count', y='price_current', trendline='ols', width=900).show()

px.scatter(sandbox, x='overall_rating', y='price_current', trendline='ols', width=900).show()

px.scatter(sandbox, x='overall_rating', y='review_count', trendline='ols', width=900).show()

px.scatter(sandbox, x='latitude', y='price_current', trendline='ols', width=900).show()

px.scatter(sandbox, x='longitude', y='price_current', trendline='ols', width=900).show()

fig = px.scatter_mapbox(sandbox, lat="latitude", lon="longitude", 
                        hover_name='loc_name',
                        hover_data=["latitude","longitude"],
                        zoom=4, height=500, width=400
                        )
fig.update_layout(mapbox_style="open-street-map")
fig.show()

fig = px.scatter_mapbox(sandbox[sandbox['price_current'] < 5], lat="latitude", lon="longitude", 
                        hover_name='loc_name',
                        hover_data=["latitude","longitude"],
                        zoom=4, height=500, width=400,
                        color='product_name'
                        )
fig.update_layout(mapbox_style="open-street-map")
fig.show()

px.scatter(sandbox, x='loc_name', y='price_current', width=900)

In [4]:
# Feature Selection (Code 19)
# Drop features
sandbox = raw.copy()

# sandbox['price_weekday'] = pd.to_datetime(sandbox['price_time_stamp'], format='%Y-%m-%d %H:%M:%S').apply(lambda i: str(i.weekday()))
# sandbox['zip2'] = sandbox['postal_code'].apply(lambda i: str(str(i)[1]))

sandbox.drop(columns=['source_url', 'DATE_SCRAPED', 'RUN_START_DATE', 
                      'zip_code_searched', 'country', 'currency', 'state',
                      'address_1', 'address_2', 'services_included', 
                      'price_time_stamp'
                      ],
             inplace=True)
sandbox = sandbox[sandbox['price_current'] != 0]
# sandbox = sandbox.dropna(how='any', axis=0)

print(sandbox.nunique())
print(sandbox.info())

# Encode and standardize (includes processing done for histograms)
s = StandardScaler()

def label_encode(df):
    le = LabelEncoder()
    for col in df.columns:
        if type(df[col][0]) is str:
            le = le.fit(df[[col]].values.ravel())
            df[col] = le.transform(df[[col]].values.ravel())

def one_hot(df, cols):
    ohe = OneHotEncoder()
    for col in cols:
        if type(df[col][0]) is str:
            ohe = ohe.fit(df[[col]])
            enc_arr = ohe.transform(df[[col]]).toarray()
            onehot_df = pd.DataFrame(enc_arr, columns=ohe.get_feature_names_out([col]))
            if len(onehot_df.columns) > 5000:
                raise MemoryError(f'There are > 5000 columns in this encoded feature ({col}). Concatenating on input dataframe is expensive and may crash. Please reduce the number of columns for this feature or reduce the number of possible values for this feature.')
            df = df.drop(columns=[col])
            df = pd.concat([df, onehot_df], axis=1)
    return df

# s = s.fit(sandbox[['services_included']])
# sandbox['services_included'] = s.transform(sandbox[['services_included']])
y_name = 'price_current'
x_name = [name for name in sandbox.columns if name != y_name]

# sandbox = one_hot(sandbox, x_name)
label_encode(sandbox)

# Drop NAs and display metadata
sandbox = sandbox.dropna(how='any', axis=0)
sandbox.info()

# Model Training (Code 20-25)

# CODE 20----------------------------------------------------------------
def train_model(df):
    y_name = 'price_current'
    x_name = [name for name in df.columns if name != y_name]
    X = df[x_name]
    y = df[y_name]

    X_t, X_v, y_t, y_v = train_test_split(X, y, test_size=0.2, random_state=42)

    lr = LinearRegression()
    lr.fit(X_t, y_t)

    weights = lr.coef_
    print('WEIGHTS---------------------')
    for i, w in zip(x_name, weights):
        print(f'{i.ljust(20)} {w}')
    print('')

    y_hat = lr.predict(X_v)
    err = y_v-y_hat
    sigma_y = stdev(y_v)
    sigma_e = stdev(err)
    mse = mean_squared_error(y_v, y_hat)
    lmbda = 1
    me = sqrt(mse)
    pe = me/(2*lmbda*sigma_y)
    performance = me/(2*lmbda*sigma_e)

    print('PERFORMANCE----------------')
    print(f'{"Mean Square Error".ljust(20)} {mse}')
    print(f'{"Mean Error".ljust(20)} {me}')
    print(f'{"Mean Percent Error".ljust(20)} {pe}')
    print(f'{"Error Variance".ljust(20)} {variance(err)}')
    print(f'{"Adj MSE Performance".ljust(20)} {performance}')

    # CODE 21----------------------------------------------------------------
    px.histogram(x=err, width=900).show()

    # CODE 22----------------------------------------------------------------
    x_p = []
    y_p = []

    for i in range(1, 5):
        x_p.append(i)
        y_p.append(me/(2*i*sigma_y))

    x_e = []
    y_e = []

    for i in range(1, 5):
        x_e.append(i)
        y_e.append(me/(2*i*sigma_e))

    fig = px.line(x=x_p, y=[y_p, y_e], labels={'x':'Lambda', 'value': 'Value'}, width=900)
    newnames = {'wide_variable_0':'MPE', 'wide_variable_1': 'Adjusted MSE'}
    fig.for_each_trace(lambda t: t.update(name = newnames[t.name],
                                          legendgroup = newnames[t.name],
                                          hovertemplate = t.hovertemplate.replace(t.name, newnames[t.name])
                                        )
                      )

    return fig, lr

fig, model = train_model(sandbox)
fig.show()
# # CODE 23----------------------------------------------------------------
# sandbox = sandbox.drop(columns=['loc_name', 'city'])
# train_model(sandbox)

# # CODE 24----------------------------------------------------------------
# sandbox = sandbox.drop(columns=['review_count'])
# train_model(sandbox)

# # CODE 25----------------------------------------------------------------
# sandbox = sandbox.drop(columns=['services_included'])
# train_model(sandbox)

# CODE 26----------------------------------------------------------------


postal_code       7100
loc_name           816
city               830
review_count       468
latitude          7774
product_name         6
payment_type         2
phone             7083
loc_number        7781
price_current      282
longitude         7772
overall_rating      39
dtype: int64
<class 'pandas.core.frame.DataFrame'>
Int64Index: 37924 entries, 0 to 53730
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   postal_code     37909 non-null  object 
 1   loc_name        37924 non-null  object 
 2   city            37924 non-null  object 
 3   review_count    37375 non-null  float64
 4   latitude        37924 non-null  float64
 5   product_name    37924 non-null  object 
 6   payment_type    37924 non-null  object 
 7   phone           35747 non-null  object 
 8   loc_number      37924 non-null  int64  
 9   price_current   37924 non-null  float64
 10  longitude       37924 non-null  float64
 11  overall_rati

In [32]:
def get_proxies():
    url = 'https://gasbuddy.com'
    proxies_url = 'https://api.proxyscrape.com/v2/?request=getproxies&protocol=http&timeout=10000&country=all&ssl=all&anonymity=all'

    response = requests.get(proxies_url)
    proxies = response.text.split('\r\n')

    working_proxies = []
    for proxy in proxies:
        if len(working_proxies) > 10:
            return working_proxies
        try:
            r = requests.get(url, proxies={'http': proxy, 'https': proxy}, timeout=5)
            check = u'\u2713'
            print(f'{check}\t Proxy {proxy} is WORKING')
            working_proxies.append(proxy)
        except:
            check = u'\u2717'
            print(f'{check}\t Proxy {proxy} is not working')
    return working_proxies
get_proxies()

✗	 Proxy 178.32.101.200:80 is not working
✗	 Proxy 51.178.47.12:80 is not working
✗	 Proxy 117.160.250.137:8080 is not working
✗	 Proxy 182.55.48.187:80 is not working
✗	 Proxy 117.160.250.138:8081 is not working
✗	 Proxy 185.143.42.54:9991 is not working
✓	 Proxy 157.90.159.179:8080 is WORKING
✗	 Proxy 198.11.172.137:80 is not working
✗	 Proxy 134.209.29.120:3128 is not working
✓	 Proxy 173.212.200.30:3128 is WORKING
✗	 Proxy 64.225.8.82:9986 is not working
✗	 Proxy 97.74.92.60:80 is not working
✗	 Proxy 159.138.252.45:3128 is not working
✗	 Proxy 3.89.91.40:80 is not working
✗	 Proxy 59.57.50.114:9091 is not working
✓	 Proxy 102.165.51.172:3128 is WORKING
✗	 Proxy 120.82.174.128:9091 is not working
✓	 Proxy 138.201.245.91:8080 is WORKING
✗	 Proxy 112.245.48.74:9002 is not working
✗	 Proxy 183.64.115.112:9091 is not working
✗	 Proxy 212.57.126.106:3128 is not working
✗	 Proxy 64.225.4.63:9993 is not working
✗	 Proxy 83.171.90.83:8080 is not working
✗	 Proxy 193.15.14.198:80 is not wor

['157.90.159.179:8080',
 '173.212.200.30:3128',
 '102.165.51.172:3128',
 '138.201.245.91:8080',
 '135.181.162.125:3128',
 '162.55.169.39:8080',
 '65.108.230.239:37559',
 '64.225.4.81:9990',
 '103.240.232.62:3128',
 '64.225.8.142:9996',
 '64.225.8.132:9998']

In [19]:
def load_page(url):
    hdr = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36',
    'Accept-Language': 'en-US,en;q=0.5'
    }    
    resp = requests.get(url, headers=hdr, proxies={'http':'102.165.51.172:3128', 'https':'102.165.51.172:3128'})
    return resp

def get_ids(zipcode):
    url = f"https://www.gasbuddy.com/home?search={zipcode}&fuel=1&method=all&maxAge=24"
    resp = load_page(url)
    soup = BeautifulSoup(resp.text, "html.parser")
    print(resp)
    u = []
    ids = soup.select('div[class*="GenericStationListItem-module__station___"]')
    for i in ids:
        u.append(i.get('id'))
    return u

def get_info(id, city):
    url = f"https://www.gasbuddy.com/station/{id}"
    resp = load_page(url)
    soup = BeautifulSoup(resp.text, "html.parser")

    features = []
    zip_match = re.search(r'CA,([0-9]{5})', str(soup.select('a[class*="Station-module__directionsLink___"]')[0].get('href')))
    postal_code = zip_match.group(1) if zip_match else None
    
    features.append(postal_code)

    try:
        loc_name = [item.text.split("\xa0")[0] for item in soup.select('h2[class*="StationInfoBox-module__header___"]')][0]
    except IndexError:
        try:
            loc_name = [item.text for item in soup.select('h2[class*="StationInfoBox-module__header___"]')]
        except IndexError:
            print('Error parsing loc_name: no stations exist in the zipcode or zipcode does not exist. Skipping...')
            return 0
    features.append(loc_name)

    features.append(city)

    review_count = int([re.split("[()]+", item.text)[1] for item in soup.select('span[class*="StationInfoBox-module__ratings___"]')][0])
    features.append(review_count)

    try:
        latitude = float(str(soup.select('a[class*="Station-module__directionsLink___"]')[0].get('href')).split('@')[1].split(',')[0])
    except:
        print(f'Error parsing latitude. Skipping...')
        return 0
    features.append(latitude)

    features.append(id)

    try:
        phone = [item.text for item in soup.select('a[class*="PhoneLink-module__blue___"]')][0]
    except IndexError:
        phone = ''
        print(f'Error parsing phone: Either phone was empty or incorrect. Skipping...')
    features.append(phone)

    try:
        longitude = float(str(soup.select('a[class*="Station-module__directionsLink___"]')[0].get('href')).split('@')[1].split(',')[1])
    except:
        print(f'Error parsing longitude. Skipping...')
        return 0
    features.append(longitude)

    payment_method = 'Credit'
    features.append(payment_method)

    try:
        overall_rating = float([item.text for item in soup.select('span[class*="Station-module__ratingAverage___"]')][0])
    except ValueError:
        return 0
    features.append(overall_rating)

    price_val = [item.text for item in soup.select('span[class*="FuelTypePriceDisplay-module__price___"]')]
    price_key = [item.text for item in soup.select('span[class*="GasPriceCollection-module__fuelTypeDisplay"]')]
    prices = {k:v for k,v in zip(price_key, price_val)}
    features.append(prices)

    return features

df_test = pd.DataFrame(columns=['postal_code', 'loc_name', 'city', 'review_count', 'latitude', 'loc_number', 'phone', 'longitude', 'payment_type', 'overall_rating', 'prices'])
zipcodes = pd.read_csv('zipcodes.us.csv', usecols=['state_code', 'zipcode', 'place'])
zipcodes = zipcodes[zipcodes['state_code'] == 'CA'].drop(columns=['state_code'])

n = 25

for k, i in enumerate(zipcodes['zipcode'].sample(n+1)):
    print(f'({k}/{n})')
    print(f'[Zip: {i}] Retrieving IDs...')
    ids = get_ids(i)
    print(f'[Zip: {i}] Done.')
    failed = False
    for j in ids:
        print(f'[Zip: {i}] Retrieving ID {j} features...')
        features = get_info(j, zipcodes.iloc[k]['place'])
        if features == 0:
            failed = True
            break
        df_test.loc[len(df_test)] = features
        print(f'[Zip: {i}] Done.')
    if failed:
        print(f'[Zip: {i}] Parse failed.')
        continue
    print(f'[Zip: {i}] Parse successful.')
    cooldown = 5
    for t in range(cooldown, 0, -1):
        print(f"\rCooldown: {t}s", end='')
        time.sleep(1)
    print(f"\rCooldown: Done.", end='')
    print('\n\n')

df_test.to_csv('gasbuddy_test.csv', index=False)

(0/25)
[Zip: 92258] Retrieving IDs...
<Response [200]>
[Zip: 92258] Done.
[Zip: 92258] Retrieving ID 36474 features...
[Zip: 92258] Done.
[Zip: 92258] Retrieving ID 139750 features...
[Zip: 92258] Done.
[Zip: 92258] Retrieving ID 39422 features...
[Zip: 92258] Done.
[Zip: 92258] Retrieving ID 36473 features...
[Zip: 92258] Done.
[Zip: 92258] Retrieving ID 39423 features...
[Zip: 92258] Done.
[Zip: 92258] Parse successful.
Cooldown: Done.


(1/25)
[Zip: 93304] Retrieving IDs...
<Response [200]>
[Zip: 93304] Done.
[Zip: 93304] Retrieving ID 98554 features...
[Zip: 93304] Done.
[Zip: 93304] Retrieving ID 201479 features...
[Zip: 93304] Done.
[Zip: 93304] Retrieving ID 106005 features...
[Zip: 93304] Done.
[Zip: 93304] Retrieving ID 98798 features...
[Zip: 93304] Done.
[Zip: 93304] Retrieving ID 125942 features...
[Zip: 93304] Done.
[Zip: 93304] Retrieving ID 71585 features...
[Zip: 93304] Done.
[Zip: 93304] Retrieving ID 119349 features...
[Zip: 93304] Done.
[Zip: 93304] Retrieving ID 121

In [20]:
# df_test_2 = pd.read_csv('gasbuddy_test.csv')
df_test_2 = df_test.copy()
product_prices = pd.DataFrame(df_test_2['prices'].tolist()).stack().reset_index(level=1).rename(columns={0:'price_current'})
product_prices['product_name'] = product_prices['level_1']
product_prices = product_prices.drop('level_1', axis=1)
df_test_2 = pd.merge(df_test_2, product_prices, left_on=df_test_2.index, how='left', right_on=product_prices.index)
df_test_2 = df_test_2.drop(columns=['prices', 'key_0'])
df_test_2 = df_test_2[df_test_2['price_current'] != '- - -']
df_test_2['price_current'] = [float(i.replace('$','')) for i in df_test_2['price_current']]
df_test_2 = df_test_2.reset_index(drop=True)
df_test_2[:5]

,postal_code,loc_name,city,review_count,latitude,loc_number,phone,longitude,payment_type,overall_rating,price_current,product_name
0,92262,Shell,Alameda,75,33.906096,36474,1 (760) 251-9096,-116.543757,Credit,3.8,4.39,Regular
1,92262,Shell,Alameda,75,33.906096,36474,1 (760) 251-9096,-116.543757,Credit,3.8,4.59,Midgrade
2,92262,Shell,Alameda,75,33.906096,36474,1 (760) 251-9096,-116.543757,Credit,3.8,4.79,Premium
3,92262,Shell,Alameda,75,33.906096,36474,1 (760) 251-9096,-116.543757,Credit,3.8,5.39,Diesel
4,92240,ARCO,Alameda,137,33.906710,139750,1 (760) 251-5741,-116.544890,Credit,4.0,4.49,Regular


In [30]:
y_name = 'price_current'
x_name = [name for name in sandbox.columns if name != y_name]

label_encode(df_test_2)

df_test_2 = df_test_2.dropna(how='any', axis=0)

print(df_test_2.info())
test_results = pd.DataFrame()
test_results['y_hat_test'] = model.predict(df_test_2[x_name])
test_results['y_test'] = df_test_2['price_current']
test_results['diff'] = test_results['y_test'] - test_results['y_hat_test']
px.histogram(test_results, x='diff', nbins=100, width=900, marginal='violin')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 570 entries, 0 to 569
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   postal_code     570 non-null    int64  
 1   loc_name        570 non-null    int64  
 2   city            570 non-null    int64  
 3   review_count    570 non-null    int64  
 4   latitude        570 non-null    float64
 5   loc_number      570 non-null    int64  
 6   phone           570 non-null    int64  
 7   longitude       570 non-null    float64
 8   payment_type    570 non-null    int64  
 9   overall_rating  570 non-null    float64
 10  price_current   570 non-null    float64
 11  product_name    570 non-null    int64  
dtypes: float64(4), int64(8)
memory usage: 53.6 KB
None
